In [3]:
import numpy as np
import torch
X = torch.rand(4, 3) # N * D = 4 * 3
W = torch.rand(3, 5) # D * C = 3 * 5
y = torch.randperm(4) # N * 1 = 4
num_train = X.shape[0]
num_classes = W.shape[1]
reg = 0.01

In [49]:
# element-wise multiclass SVM
dW_elem = torch.zeros_like(W)
loss_elem = 0

for i in range(num_train):
    #loss_elem = 0
    score_elem = W.t().mv(X[i])
    # svm is max(0, 1 + score[j] - correct_score[y[i]])
    for j in range(num_classes):
        if (j == y[i]):
            continue
        margin = 1 + score_elem[j] - score_elem[y[i]]
        #print(margin, end='')
        if margin > 0:
            loss_elem += margin
            dW_elem[:, j] += X[i, :]
            dW_elem[:, y[i]] += -1 * X[i, :]
    #print('\n')
loss_elem /= num_train
loss_elem += reg * torch.sum(W * W)
dW_elem /= num_train
dW_elem += 2 * reg * W
print('dW = ', dW_elem, '\n')
print('loss =', loss_elem)



dW =  tensor([[-0.2438,  0.1290, -0.5054,  0.0358,  0.6338],
        [-0.4746,  0.4525, -0.5251,  0.1025,  0.5000],
        [ 0.2970,  0.3109, -0.3589, -0.5410,  0.3348]]) 

loss = tensor(3.3617)


In [19]:
b = torch.arange(num_train)
score_vec = X.mm(W)
print(score_vec, '\n')
print(score_vec[b, y].reshape(-1, 1))
print(torch.sum(score_vec, axis = 1))

tensor([[0.9110, 1.0863, 1.8981, 0.9080, 1.9733],
        [0.9517, 1.2455, 2.0831, 0.9428, 2.1483],
        [0.8564, 1.0001, 1.7654, 0.8522, 1.8394],
        [0.4191, 0.5451, 0.9120, 0.4518, 0.9290]]) 

tensor([[1.8981],
        [0.9517],
        [1.0001],
        [0.4518]])
tensor([6.7769, 7.3714, 6.3136, 3.2570])


In [47]:
# vector-wise multiclass SVM
dW_vec = torch.zeros_like(W) # D * C = 3 * 5
loss_vec = 0
b = torch.arange(num_train) # N * 1 = 4

score_vec = X.mm(W) # N * C
correct_score = score_vec[b, y]

margin_vec = torch.maximum(torch.zeros_like(score_vec), score_vec - correct_score.reshape(-1, 1) + 1)
margin_vec[b, y] = 0
loss_vec += torch.sum(margin_vec)
loss_vec /= num_train
loss_vec += reg * torch.sum(W * W)

# 2, not correct class, margin > 0 dW[:, j] = sum X[i, :], so dW each row are same if linear
# 3, not correct class, margin <=0 dW = 0
# svm: for each batch of N, incorrect class is score - correct_score
binary = margin_vec
print(margin_vec)
binary[margin_vec > 0] = 1 # here margin and binary changed together!!!
#print(margin_vec)

# 1, correct score dW = -1
each_row = torch.sum(binary, axis = 1) # sum up all columns for each row
# fill all entries of binary
binary[b, y] = -each_row
dW_vec = X.t().mm(binary) # D * C = (D * N) * (N * C)

dW_vec /= num_train
dW_vec += reg * 2 * W

#print(margin_vec)
#print(binary)
print("dW = ", dW_vec, '\n')
print("loss =", loss_vec)

tensor([[0.0000, 0.3654, 0.0551, 0.5661, 0.1929],
        [1.1504, 0.5004, 0.5943, 0.0000, 0.4265],
        [1.0290, 0.0000, 0.7166, 0.8194, 0.8528],
        [2.1294, 1.1793, 0.0000, 1.6837, 0.9860]])
dW =  tensor([[-0.2438,  0.1290, -0.5054,  0.0358,  0.6338],
        [-0.4746,  0.4525, -0.5251,  0.1025,  0.5000],
        [ 0.2970,  0.3109, -0.3589, -0.5410,  0.3348]]) 

loss = tensor(3.3617)
